# Vector Space Model
_to implement "Twitter Docs Search Engine"_

## Aquiring Data

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
tweets_df = pd.read_csv(r'../scraper/tweets.csv')
tweets_df.fillna(0)

,User,Handle,PostDate,TweetText,ReplyCount,RetweetCount,LikeCount
0,Kimberly Lantis,@KimLantis,2021-09-30T15:54:03.000Z,Hey \n@dojolive\n fans! We have a new podcast ...,0.0,0.0,0.0
1,Ryan Hurst,@rmhrisk,2021-09-30T15:54:03.000Z,We are dependant on a number of ancient techno...,0.0,0.0,0.0
2,ملاك حداد,@ivbaoshi,2021-09-30T15:54:01.000Z,All this technology and the public forget that...,0.0,0.0,0.0
3,GalaxyOne,@Galaxy_One1,2021-09-30T15:54:01.000Z,What is a NFT?\n\n#technologies #Technologie #...,0.0,0.0,0.0
4,Antifa #SquirrelSquad Officer,@SquizzRadical,2021-09-30T15:54:00.000Z,We already created God. Think we godda work on...,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1854,ARSLAN|ARTAH|,@fgfsh1633010846,2021-09-30T14:10:54.000Z,"Country to least have a lot, they\nPeople comm...",0.0,0.0,1.0
1855,RPML Technology Pvt Ltd,@dnshkmr73,2021-09-30T14:10:54.000Z,We have team of better experienced developers ...,0.0,0.0,0.0
1856,Mud Cat Dredges,@MudCatDredge,2021-09-30T14:10:52.000Z,Two Mud Cat E-Series Dredges were started up l...,0.0,0.0,1.0
1857,RjTradingStocks,@rjstocks3,2021-09-30T14:10:52.000Z,Replying to \n@HKHabibKhan7\n\nAre they being ...,0.0,1.0,2.0


In [3]:
tweet_text = pd.Series(tweets_df['TweetText'])
tweet_text

0       Hey \n@dojolive\n fans! We have a new podcast ...
1       We are dependant on a number of ancient techno...
2       All this technology and the public forget that...
3       What is a NFT?\n\n#technologies #Technologie #...
4       We already created God. Think we godda work on...
                              ...                        
1854    Country to least have a lot, they\nPeople comm...
1855    We have team of better experienced developers ...
1856    Two Mud Cat E-Series Dredges were started up l...
1857    Replying to \n@HKHabibKhan7\n\nAre they being ...
1858    Hey, do you want to know what happened in the ...
Name: TweetText, Length: 1859, dtype: object

In [4]:
tweet_text.isnull().sum()

0

In [5]:
length = len(tweet_text)
length

1859

## Text Pre-processing Functions

In [6]:
!pip3 install num2words

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from num2words import num2words

stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

In [8]:
def to_lower(data):
    '''Lowercases the dataset documents'''
    op_data = []
    for item in data:
        item = item.lower()
        op_data.append(item)
    return op_data

def q_to_lower(q):
    '''Lowercases the query'''
    return q.lower()

In [9]:
def remove_stopwords(data):
    '''Removes Stopwords from the dataset documnets'''
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stopwords.words('english'):
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

def q_remove_stopwords(q):
    '''Removes stopwords from the query'''
    stop_words = stopwords.words('english')
    words = word_tokenize(str(q))
    new_q = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_q = new_q + " " + w
    return new_q

In [10]:
def clean_symbols(data):
    '''Removes special charecters and punctuations from the dataset documents'''
    symbols = "!\"#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"+"'"
    clean_data = []

    for item in data:
        for i in symbols:
            # print('before: ', item)
            item = str(np.char.replace(item, i, ' '))
            # print('after: ', item)
        clean_data.append(item)
        
    return clean_data

def q_clean_symbols(q):
    '''Removes special charecters and punctuations from the query'''
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        q = np.char.replace(q, symbols[i], ' ')
        q = np.char.replace(q, "  ", " ")
    q = np.char.replace(q, ',', '')
    q = np.char.replace(q, "'", '')
    return q

In [11]:
def stem(data):
    '''
        Performs stemming on the dataset documents.
        Essentially, converts every word to its root form.
    '''
    for item in data:
        for word in item:
            # print("before lemma:", word)
            word = stemmer.stem(word)
            # print("after lemma:", word)
    return data

# def lemmatize(data):
#     for item in data:
#         for word in item:
#             # print("before lemma:", word)
#             word = lemmatizer.lemmatize(word)
#             # print("after lemma:", word)
#     return data

def q_stem(q):
    '''
        Performs stemming on the query.
        Essentially, converts every word to its root form.
    '''
    tokens = word_tokenize(str(q))
    new_q = ""
    for w in tokens:
        new_q = new_q + " " + stemmer.stem(w)
    return new_q

# def q_lemmatize(q):
#     tokens = word_tokenize(str(q))
#     new_q = ""
#     for w in tokens:
#         new_q = new_q + " " + lemmatizer.lemmatize(w)
#     return new_q

In [12]:
def preprocess(data):
    '''Applies all the pre-processing methods on the dataset documents'''
    data = pd.Series(to_lower(data))
    data = pd.Series(clean_symbols(data) )
    data = pd.Series(remove_stopwords(data))
    data = stem(data)
    data = pd.Series(clean_symbols(data))
    data = pd.Series(remove_stopwords(data))
    return data

def q_preprocess(q):
    '''Applies all the pre-processing methods on the query'''
    q = q_to_lower(q)
    q = q_clean_symbols(q)
    q = q_remove_stopwords(q)
    q = q_stem(q)
    q = q_clean_symbols(q)
    q = q_remove_stopwords(q)
    return q

In [13]:
def tokenize(data):
    '''Tokenizes the strings in the input list'''
    tkns = []

    for item in data:
        tkns.append(word_tokenize(item))
    
    return tkns

## Vectorization and TF-IDF

In [14]:
tweets = preprocess(tweet_text)
tweets

0       hey dojolive fans new podcast available spotif...
1       dependant number ancient technologies make mod...
2       technology public forget people behind good sh...
3       nft technologies technologie technologynews te...
4       already created god think godda work associati...
                              ...                        
1854    country least lot people commonly proper persp...
1855    team better experienced developers working dja...
1856    two mud cat e series dredges started last week...
1857    replying hkhabibkhan7 led developing team tech...
1858    hey want know happened anti vaccine beliefs te...
Length: 1859, dtype: object

In [15]:
tweet_tokens = tokenize(tweets)
tweet_tokens

[['hey',
  'dojolive',
  'fans',
  'new',
  'podcast',
  'available',
  'spotify',
  'get',
  'listening',
  'http',
  'dojo',
  'live',
  'dojolive',
  'technology',
  'innovation',
  'culture',
  'podcast',
  'newpodcast'],
 ['dependant',
  'number',
  'ancient',
  'technologies',
  'make',
  'modern',
  'computing',
  'work',
  'solid',
  'ip',
  'others',
  'due',
  'replacement',
  'ones',
  'fall',
  'second',
  'category'],
 ['technology',
  'public',
  'forget',
  'people',
  'behind',
  'good',
  'shady',
  'personalities'],
 ['nft',
  'technologies',
  'technologie',
  'technologynews',
  'technologytrends',
  'computertechnology',
  'computer',
  'informationtechnology',
  'information',
  'helpful',
  'technologytips',
  'simpleknowledge'],
 ['already',
  'created',
  'god',
  'think',
  'godda',
  'work',
  'associating',
  'technology',
  'like',
  'zachery',
  'tyson',
  'zaknafeindc',
  '·',
  '48m',
  'could',
  'roll',
  'eyes',
  'hard',
  'enough',
  'stupidity',
  

In [16]:
# Calculating Data Frequency
DF = {}

for i in range(length):
    tokens = tweet_tokens[i]
    for token in tokens:
        try:
            DF[token].add(i)
        except:
            DF[token] = {i}

for i in DF:
    DF[i] = len(DF[i])
    
DF

{'hey': 7,
 'dojolive': 1,
 'fans': 2,
 'new': 242,
 'podcast': 17,
 'available': 27,
 'spotify': 1,
 'get': 67,
 'listening': 2,
 'http': 125,
 'dojo': 1,
 'live': 24,
 'technology': 1342,
 'innovation': 53,
 'culture': 15,
 'newpodcast': 1,
 'dependant': 1,
 'number': 10,
 'ancient': 5,
 'technologies': 285,
 'make': 41,
 'modern': 24,
 'computing': 8,
 'work': 54,
 'solid': 6,
 'ip': 3,
 'others': 79,
 'due': 17,
 'replacement': 2,
 'ones': 4,
 'fall': 5,
 'second': 10,
 'category': 5,
 'public': 23,
 'forget': 3,
 'people': 93,
 'behind': 26,
 'good': 36,
 'shady': 1,
 'personalities': 1,
 'nft': 21,
 'technologie': 1,
 'technologynews': 4,
 'technologytrends': 1,
 'computertechnology': 1,
 'computer': 18,
 'informationtechnology': 2,
 'information': 50,
 'helpful': 2,
 'technologytips': 1,
 'simpleknowledge': 1,
 'already': 16,
 'created': 11,
 'god': 2,
 'think': 48,
 'godda': 1,
 'associating': 1,
 'like': 81,
 'zachery': 1,
 'tyson': 1,
 'zaknafeindc': 1,
 '·': 101,
 '48m': 1,


In [17]:
vocabulary = [x for x in DF]
vocabulary

['hey',
 'dojolive',
 'fans',
 'new',
 'podcast',
 'available',
 'spotify',
 'get',
 'listening',
 'http',
 'dojo',
 'live',
 'technology',
 'innovation',
 'culture',
 'newpodcast',
 'dependant',
 'number',
 'ancient',
 'technologies',
 'make',
 'modern',
 'computing',
 'work',
 'solid',
 'ip',
 'others',
 'due',
 'replacement',
 'ones',
 'fall',
 'second',
 'category',
 'public',
 'forget',
 'people',
 'behind',
 'good',
 'shady',
 'personalities',
 'nft',
 'technologie',
 'technologynews',
 'technologytrends',
 'computertechnology',
 'computer',
 'informationtechnology',
 'information',
 'helpful',
 'technologytips',
 'simpleknowledge',
 'already',
 'created',
 'god',
 'think',
 'godda',
 'associating',
 'like',
 'zachery',
 'tyson',
 'zaknafeindc',
 '·',
 '48m',
 'could',
 'roll',
 'eyes',
 'hard',
 'enough',
 'stupidity',
 'show',
 'thread',
 'innovative',
 'ultimate',
 'cleaning',
 'performance',
 'clever',
 'water',
 'distribution',
 'system',
 'bristle',
 'precision',
 'last',
 

In [18]:
vocab_size = len(vocabulary)
vocab_size

14042

In [19]:
def doc_frequency(word):
    """This function return the Document frequency of the word that is input in."""
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [20]:
doc = 0
tf_idf = {}

for i in range(length):
    tokens = tweet_tokens[i]
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_frequency(token)
        idf = np.log((length+1)/(df+1))
        tf_idf[doc, token] = tf*idf

    doc += 1
    
tf_idf

{(0, 'available'): 0.23311818091844683,
 (0, 'culture'): 0.2642079469148592,
 (0, 'dojo'): 0.3797324770081834,
 (0, 'dojolive'): 0.7594649540163668,
 (0, 'fans'): 0.357206637668841,
 (0, 'get'): 0.18382355897395222,
 (0, 'hey'): 0.30271612361263395,
 (0, 'http'): 0.14955832554198717,
 (0, 'innovation'): 0.1966304288968318,
 (0, 'listening'): 0.357206637668841,
 (0, 'live'): 0.23941421899105814,
 (0, 'new'): 0.11307057352037213,
 (0, 'newpodcast'): 0.3797324770081834,
 (0, 'podcast'): 0.515328889867898,
 (0, 'spotify'): 0.3797324770081834,
 (0, 'technology'): 0.018092809454667184,
 (1, 'ancient'): 0.33744542926348187,
 (1, 'category'): 0.33744542926348187,
 (1, 'computing'): 0.3135945405512369,
 (1, 'dependant'): 0.40206968153807654,
 (1, 'due'): 0.2728211769888872,
 (1, 'fall'): 0.33744542926348187,
 (1, 'ip'): 0.3612963179757268,
 (1, 'make'): 0.2229801263778752,
 (1, 'modern'): 0.25349740834347334,
 (1, 'number'): 0.30179038199463976,
 (1, 'ones'): 0.34817022672194975,
 (1, 'others')

## Ranking the Documents using Matching Score and Cosine Similarity

In [21]:
from termcolor import colored

In [23]:
def matching_score(k, query):
    '''Performs the Matching Score Technique to Rank the documents'''
    pp_query = q_preprocess(query)
    tokens = word_tokenize(str(pp_query))

    print("Matching Score")
    print("\nQuery:", pp_query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    out = []
    for i in query_weights[:k]:
        out.append(i[0])
    
    print(out)
    
    for num,i in zip(out, range(k)):
        print(colored(f"#{i+1} ->", "red"), colored(tweet_text[num], "blue"), end="\n\n")
    

# matching_score(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

In [24]:
def cosine_sim(a, b):
    '''Calculates the Cosine between the two vectors a and b'''
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [25]:
D = np.zeros((length, vocab_size))
for i in tf_idf:
    try:
        ind = vocabulary.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [26]:
def vectorize(tokens):
    '''Vectorizes the tokens given as parameters'''
    Q = np.zeros(vocab_size)
    counter = Counter(tokens)
    words_count = len(tokens)
    query_weights = {}
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_frequency(token)
        idf = math.log((length+1)/(df+1))

        try:
            ind = vocabulary.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [27]:
def cosine_similarity(k, query):
    '''Performs the Cosine Similarity Technique to Rank the documents'''
    print("Cosine Similarity")
    pp_query = q_preprocess(query)
    tokens = word_tokenize(str(pp_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cos = []
    q_vec = vectorize(tokens)
    
    for d in D:
        d_cos.append(cosine_sim(q_vec, d))
        
    out = np.array(d_cos).argsort()[-k:][::-1]
    
    print("")
    
    print(out)
    
    for num,i in zip(out, range(k)):
        print(colored(f"#{i+1} ->", "red"), colored(tweet_text[num], "blue"), end="\n\n")

# query = cosine_similarity(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

## User Interface

In [28]:
!pip3 install ipywidgets

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [29]:
import ipywidgets as widgets

In [30]:
query = widgets.Text(
    value='',
    placeholder='Search Here',
    description= "Query",
    disabled=False
)

In [31]:
n = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='No. of Docs:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [32]:
technique = widgets.RadioButtons(
    options=['Matching Score', 'Cosine Similarity'],
    value='Cosine Similarity',
    layout={'width': 'max-content'},
    description='Rank Algo:',
    disabled=False
)

In [33]:
def search(arg):
    if technique.value=="Matching Score":
        matching_score(n.value, query.value)
    elif technique.value=="Cosine Similarity":
        cosine_similarity(n.value, query.value)

In [34]:
button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='success',
    icon='search'
)

button.on_click(search)

In [35]:
out = widgets.GridBox([query, n, technique, button])

---------

In [36]:
out

GridBox(children=(Text(value='', description='Query', placeholder='Search Here'), IntSlider(value=10, continuo…